In [1]:
%reset -f
# import the necessary packages
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Activation, Flatten, Dense, GlobalAveragePooling2D, Dropout
from tensorflow.keras import backend as K
from tensorflow.keras.callbacks import ModelCheckpoint, TensorBoard, EarlyStopping, ReduceLROnPlateau, CSVLogger
from tensorflow.keras.applications.vgg16 import VGG16
from tensorflow.keras.applications.inception_v3 import InceptionV3
from tensorflow.keras.applications.densenet import DenseNet121, DenseNet201
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.optimizers import Adam
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical
from sklearn.metrics import confusion_matrix, balanced_accuracy_score
import seaborn as sns
from sklearn import preprocessing
from sklearn.utils import shuffle
from sklearn.preprocessing import LabelEncoder
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import argparse
import random
import cv2, glob
import os
import numpy as np
seed = 42
np.random.seed(seed)
import os, shutil, glob, cv2, sys, argparse
from natsort import natsorted
from insectrec.utils import *
from tqdm import tqdm
# sns.set()
seed = 42
np.random.seed(seed)
batch_size = 32
nb_classes = 9

created_data_path = '/home/kalfasyan/data/images/sticky_plates/created_data'
path_impy_crops_export = f'{created_data_path}/impy_crops_export/'
path_images_augmented = f'{created_data_path}/images_augmented/'
img_dim = 150
modelname = 'model_platebased_DENSE121_dropout05_150.h5'

from tensorflow.keras.applications import NASNetLarge

base_model = DenseNet121(include_top=False, weights=None, 
                         input_shape=(img_dim,img_dim,3))
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dropout(0.5)(x)
predictions = Dense(nb_classes, activation='softmax')(x)
model = Model(inputs=base_model.input, outputs=predictions)
# model.summary()

import math
top_weights_path = f"{created_data_path}/weights/{modelname}"
model.load_weights(top_weights_path)
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [2]:
# pred = model.predict(valid_generator(X_test, y_test, batch_size=batch_size, nb_classes=nb_classes, img_dim=img_dim), 
#                          steps= len(X_test) / batch_size, 
#                          verbose=1)

In [3]:
datadir = '/home/kalfasyan/data/images/sticky_plates/'
years = ['2020']

In [4]:
created_data_path = f'{datadir}/created_data'

# CREATING NECESSARY DIRECTORIES FOR THE PROJECT
path_annotations = f'{created_data_path}/annotations/'
path_images = f'{created_data_path}/images/'
path_voc_annotations = f'{created_data_path}/voc_annotations/'
path_crops_export = f'{created_data_path}/crops_export/'
path_images_augmented = f'{created_data_path}/images_augmented/'
path_weights = f'{created_data_path}/weights/'
path_logs = f'{created_data_path}/logs/'
for path in [created_data_path, path_annotations, path_images, path_voc_annotations, 
            path_crops_export, path_weights, path_logs, path_images_augmented]:
    if not os.path.isdir(path):
        os.mkdir(path)	

include_herent = True
yolo_to_voc = True # In the end of the script, yolo annotations get converted to voc
extract_boxes = True # Only works if above is true. Bounding boxes extracted and saved as images
clean = True # Deleting previous data created here (i.e. except of logs and weights)
save_extractions = True # Whether to save the extracted insect crops 

if clean:
    print(f'Cleaning directories..')
    clean_folder(path_annotations)
    clean_folder(path_images)
    clean_folder(path_voc_annotations)
    os.system(f'rm -rf {path_crops_export}*')
    os.system(f'rm -rf {path_images_augmented}*')
    os.system(f'rm {created_data_path}/df_*')
    os.system(f'rm {created_data_path}/class_mapping.csv')


Cleaning directories..


In [5]:
plates = []
for y in years:
	y_plates = get_plate_names(y, base_dir=datadir)
	plates += y_plates
	print(f"Number of plates: {len(y_plates)} for year: {y}")

Number of plates: 212 for year: 2020


In [6]:
labview_cols = ['Center of Mass X.1', 'Center of Mass Y.1', 'Bounding Rect Left.1',
       'Bounding Rect Top.1', 'Bounding Rect Right.1',
       'Bounding Rect Bottom.1', 'Equivalent Ellipse Major Axis.1',
       'Equivalent Ellipse Minor Axis.1', 'Area.1', 'Convex Hull Area.1',
       'Orientation.1', 'Ratio of Equivalent Ellipse Axes.1',
       'Ratio of Equivalent Rect Sides.1', 'Elongation Factor.1',
       'Compactness Factor.1', 'Heywood Circularity Factor.1', 'Type Factor.1',
       'R', 'G', 'B']
all_specs = []

In [7]:
# Loop through all plates and nested loop through all insects in the plates
for p, platename in tqdm(enumerate(plates)):

    # Defining the platename
    pname = platename.split('/')[-1][:-4] 
    if 'empty' in pname:
        continue

    # Reading the plate image
    plate_img = read_plate(platename) 
    H,W,_ = plate_img.shape

    # Reading the specifications of the plate
    spec = pd.read_csv(plates[p][:-4] + '.txt', sep="\t") 
    # Fetching column names (only needed once)
    if p == 0: 
        columns = [ii for ii in spec.columns if ii.endswith('.1')]
        colextensions = ['index', 'name plate', 'R','G','B']
        columns.extend(colextensions) # adding extra columns
    spec = spec[columns]
    spec.rename(columns={'index': 'insect_idx'}, inplace=True)
    spec.dropna(axis=0, how='any', inplace=True)

    # # ADDING YOLO AND HUMAN-READABLE ANNOTATION TO COLUMNS
    # cmap = class_map[class_map['platename'] == pname].drop_duplicates(subset='idx', keep='first')
    # sub_map = cmap[['idx','class_encoded']].set_index('idx')
    # sub_map2 = cmap[['idx','class']].set_index('idx')
    # spec['yolo_class'] = sub_map
    # spec['normal_class'] = sub_map2

    # Replacing commas from labview columns with dots
    for col in labview_cols:
        if spec[col].dtype not in [pd.np.dtype('float64')]:
            spec[col] = spec[col].str.replace(",",".").astype(float)

    print(f"Platename: {platename.split('/')[-1]}")
    spec['pname'] = pname
    # Making extracted boxes squares (to avoid distortions in future resizing)
    spec['width'] = 150
    spec['height'] = 150

    # Creating specifications according to 'YOLO' format
    spec['yolo_x'] = np.abs(spec['Bounding Rect Right.1'] - np.abs(spec['Bounding Rect Left.1'] - spec['Bounding Rect Right.1']) /2) / W
    spec['yolo_y'] = np.abs(spec['Bounding Rect Bottom.1'] - np.abs(spec['Bounding Rect Top.1'] - spec['Bounding Rect Bottom.1']) /2) / H
    spec['yolo_width'] = pd.concat([spec['width'], spec['height']], axis=1).max(axis=1) / W 
    spec['yolo_height'] = pd.concat([spec['width'], spec['height']], axis=1).max(axis=1) / H

    ann_full_new = os.path.join( path_annotations , f"{pname}.txt" )
    img_full_new = os.path.join( path_images , pname ) + '.jpg'

    # SAVING IMAGES
    if not os.path.isfile( img_full_new ):
        cv2.imwrite(img_full_new, plate_img)

    if save_extractions:
        spec['normal_class'] = f"{platename.split('/')[-4]}/{pname}"
        spec['yolo_class'] = 0
        save_insect_crops(spec, path_crops_export, plate_img)

        pltnames = pd.Series(os.listdir(f"{path_crops_export}/{platename.split('/')[-4]}/{pname}/"))
        ser = pltnames.apply(lambda x: f"{path_crops_export}/{platename.split('/')[-4]}/{pname}/{x}")
        y = [0 for i in range(len(ser))]
        
        pred = model.predict(valid_generator(ser.tolist(), y, batch_size=batch_size, nb_classes=nb_classes, img_dim=img_dim), 
                         steps= len(ser.tolist()) / batch_size, 
                         verbose=1)
        spec['possible_wmv'] = (np.round(pred[:,-1],decimals=2) * 100).astype(int)
        spec['possible_wmv'] = spec['possible_wmv'].apply(lambda x: "" if x<51 else f"wmv?")#_{x}%")
        spec['Klasse'] = ' '

    spec = spec[['pname','insect_idx','Klasse','possible_wmv']]
    spec.rename(columns={'pname': 'name plate', 'insect_idx': 'index'}, inplace=True)
    
    savedir = f"{path_crops_export}/{platename.split('/')[-4]}"
    if not os.path.isdir(savedir):
        os.makedirs(savedir)
    spec.to_csv(f"{savedir}/{pname}_specs.csv")
    all_specs.append(spec)

df_specs = pd.concat(all_specs, axis=0)

# SAVING DATAFRAMES WITH STATISTICS REGARDING THE PLATES AND BOUNDING BOXES
df_specs.to_csv(f'{created_data_path}/df_specs.csv')
print(f"path images: {path_images}")
print(f"path voc annotations: {path_voc_annotations}")

0it [00:00, ?it/s]/home/kalfasyan/anaconda3/envs/objdetect/lib/python3.7/site-packages/ipykernel_launcher.py:33: FutureWarning: The pandas.np module is deprecated and will be removed from pandas in a future version. Import numpy directly instead


Platename: HerentCONTROLE_w04_F10_51 mm_ISO160_1-15 s_IncandescentWB.png
1/0 [==============================] - 3s 3s/step


1it [00:03,  3.96s/it]

Platename: HerentCONTROLE_w05_F10_51 mm_ISO160_1-15 s_IncandescentWB.png
2/1 [==========================================] - 2s 1s/step


2it [00:06,  3.66s/it]

...

Platename: racour_20200914_B_F10_51 mm_ISO160_1-15 s_IncandescentWB.png
5/4 [==================================] - 0s 32ms/step


211it [04:37,  1.02it/s]

Platename: racour_20200914_C_F10_51 mm_ISO160_1-15 s_IncandescentWB.png
4/3 [=================================] - 0s 33ms/step


212it [04:38,  1.31s/it]

path images: /home/kalfasyan/data/images/sticky_plates//created_data/images/
path voc annotations: /home/kalfasyan/data/images/sticky_plates//created_data/voc_annotations/
